In [67]:
import numpy

In [68]:
# model parameters:
g = 9.81                          # m/s^2     - acceleration due to gravity
shell_mass = 50                   # kg        - mass of the rocket shell
air_density = 1.091               # kg/m^3    - average air density (assumed constant)
radius = 0.5                      # m         - maximum radius of the rocket
A = numpy.pi * radius**2          # m^2       - maximum cross sectional area of the rocket
exhaust_velocity = 325            # m/s       - velocity of the rocket exhaust
drag_coefficient = 0.15           # unitless
initial_propellant_mass = 100     # kg        - initial mass of the propellant
propellant_mass_burn_rate_0 = 20  # kg/s      - propellant mass burn rate

In [69]:
def get_propellant_mass(t):
    """Returns the mass of the remaining propellant.
    
    Parameters
    ----------
    t : float
        time.
    
    Returns
    -------
    propellant_mass : float
                      mass of the remaining propellant
    """
    
    if t < 5:
        propellant_mass = initial_propellant_mass - propellant_mass_burn_rate_0 * t
    else:
        propellant_mass = initial_propellant_mass - propellant_mass_burn_rate_0 * 5
    
    return propellant_mass

In [70]:
def get_propellant_mass_burn_rate(t):
    """Returns the propellant mass burn rate.
    
    Parameters
    ----------
    t : float
        time.
    
    Returns
    -------
    propellant_mass_burn_rate : float
                                propellant mass burn rate
    """
    
    if t < 5:
        propellant_mass_burn_rate = propellant_mass_burn_rate_0
    else:
        propellant_mass_burn_rate = 0
    
    return propellant_mass_burn_rate

In [71]:
def f(u, t):
    """Returns the right-hand-side of the rocket system of equations.
    
    Parameters
    ----------
    u : array of float
        array containing the solution at time n.
    t : float
        time.
    
    Returns
    -------
    u_prime : array of float
              array containing the RHS given u, t.
    """
    
    v = u[1]
    
    propellant_mass = get_propellant_mass(t)
    propellant_mass_burn_rate = get_propellant_mass_burn_rate(t)
    
    h_prime = v
    
    v_prime = ( -(shell_mass + propellant_mass) * g + \
                propellant_mass_burn_rate * exhaust_velocity - \
                (1/2) * air_density * v * abs(v) * A * drag_coefficient ) / \
              (shell_mass + propellant_mass)
    
    u_prime = numpy.array([h_prime, v_prime])
    
    return u_prime

In [72]:
def euler_step(u, f, t, dt):
    """Returns the solution at the next time-step using Euler's method.
    
    Parameters
    ----------
    u : array of float
        solution at the previous time-step.
    f : function
        function to compute the right-hand-side of the system of equations.
    t : float
        time.
    dt : float
         time-increment.
    
    Returns
    -------
    u_n_plus_1 : array of float
        approximate solution at the next time step.
    """
    
    return u + dt * f(u, t)

In [73]:
# time parameters
initial_t = 0.0
dt = 0.1
t_max = 1000.0

# initial conditions
initial_h = 0.0
initial_v = 0.0

initial_u = numpy.array([initial_h, initial_v])

t_list = [initial_t,]
h_list = [initial_h,]
v_list = [initial_v,]

In [74]:
t = initial_t
u = initial_u

counter = 0

while True:
    counter = counter + 1
    # t_next = t + dt
    # floating point arithmetic sensitivity
    # 0.1 + 0.1 + ... + 0.1 ~= 1.0
    t_next = initial_t + counter * dt
    
    if t_next > t_max:
        break
    
    u_next = euler_step(u, f, t, dt)
    h_next = u_next[0]
    v_next = u_next[1]
    
    if h_next < 0:
        break
    
    t_list.append(t_next)
    h_list.append(h_next)
    v_list.append(v_next)
    
    t = t_next
    u = u_next

In [75]:
t_array = numpy.array(t_list)
h_array = numpy.array(h_list)
v_array = numpy.array(v_list)

Question 1:
Remaining propellant mass at time t = 3.2s:

In [76]:
propellant_mass = get_propellant_mass(3.2)
print("Remaining propellant mass at time t = 3.2s: {:.2f} kg".format(propellant_mass))

Remaining propellant mass at time t = 3.2s: 36.00 kg


Question 2:
Maximum speed of the rocket:

In [77]:
max_speed = v_array.max()
print("Maximum speed: {:.2f} m/s".format(max_speed))

Maximum speed: 232.11 m/s


Question 3:
Time of maximum speed:

In [78]:
idx1 = numpy.argmax(v_array)
time_of_max_speed = t_array[idx1]
print("Time of maximum speed: {:.2f} s".format(time_of_max_speed))

Time of maximum speed: 5.00 s


Question 4:
Altitude at maximum speed:

In [79]:
altitude_at_maximum_speed = h_array[idx1]
print("Altitude at maximum speed: {:.2f} m".format(altitude_at_maximum_speed))

Altitude at maximum speed: 523.52 m


Question 5:
Maximum altitude:

In [80]:
max_altitude = h_array.max()
print("Maximum altitude: {:.2f} m".format(max_altitude))

Maximum altitude: 1334.18 m


Question 6:
Time of maximum altitude:

In [81]:
idx2 = numpy.argmax(h_array)
time_of_max_altitude = t_array[idx2]
print("Time of maximum altitude: {:.2f} s".format(time_of_max_altitude))

Time of maximum altitude: 15.70 s


Question 7:
Impact time:

In [82]:
impact_time = t_array[-1]
print("Time of impact: {:.2f} s".format(impact_time))

Time of impact: 37.00 s


Question 8:
Velocity at impact:

In [83]:
impact_velocity = v_array[-1]
print("Impact velocity: {:.2f} m/s".format(impact_velocity))

Impact velocity: -85.98 m/s
